# Priprava okolja

## Lokalna virtualka

## Google Compute Engine (GCE)

### Create a GCE Account

<p>A Google Cloud Platform account is required for this course. You can use an existing GCP account or <a target="_blank" href="https://console.cloud.google.com/freetrial">sign up for a new one</a> with a valid Gmail account.</p>

A credit card is required for Google Cloud Platform, but there is no subscription cost, you pay only for resources you use.

### Create a Project

A GCP project is required for this course. You can use an existing GCP project or create a new one.

Your project name maybe different from your project id.

### Enable Compute Engine and Container Engine APIs

In order to create the cloud resources required by this workshop, you will need to enable the following APIs using the Google API Console:
- Compute Engine API
- Container Engine API

### Enable and explore Cloud Shell

Google Cloud Shell provides you with command-line access to computing resources hosted on Google Cloud Platform and is available now in the Google Cloud Platform Console. Cloud Shell makes it easy for you to manage your Cloud Platform Console projects and resources without having to install the Google Cloud SDK and other tools on your system. With Cloud Shell, the Cloud SDK gcloud command and other utilities you need are always available when you need them.

- [Google Cloud Shell documentation](https://cloud.google.com/shell/docs)
- [Google Cloud Shell getting started guide :](https://classroom.udacity.com/courses/ud615/lessons/7826112332/concepts/81473137730923)
    - Preview and deploy a Kubernetes Engine application
    - Preview and deploy an App Engine application
    - Running gcloud commands with Cloud Shell

#### Configure Your Cloud Shell Environment

Create two Cloud Shell Sessions and run the following commands to avoid setting the compute zone.

List available time zones:

    gcloud compute zones list

Set a time zone example:

    gcloud config set compute/zone europe-west1-d